# OLS: Prezzo elettricità vs Irradiazione solare (Italia)

In questo notebook mostriamo come:
1. recuperare i dati di prezzo da Yahoo Finance (proxy del mercato elettrico);
2. recuperare i dati di irradiazione giornaliera dalla NASA POWER API;
3. unire i due dataset per data;
4. stimare una regressione OLS con `statsmodels`;
5. visualizzare i risultati con uno scatter plot e la retta di regressione.

L'obiettivo è avere un esempio completo da usare nel corso di *Regression and Time Series*.

## 1. Import dei pacchetti
Importiamo le librerie necessarie. Se manca qualcosa (`yfinance`, ad esempio) installala prima con `pip install yfinance`.

In [ ]:
import requests
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt

: 

## 2. Parametri del caso di studio
Qui fissiamo:
- le coordinate (Padova);
- il periodo (tutto il 2024);
- il ticker Yahoo Finance da usare come proxy.

Se vuoi cambiare città o periodo basta modificare queste variabili.

In [ ]:
LAT = 45.406  # Padova
LON = 11.876
START_DATE = "2024-01-01"
END_DATE = "2024-12-31"
YF_TICKER = "ELEC.F"  # proxy del prezzo elettrico europeo

## 3. Download dei prezzi da Yahoo Finance
Usiamo `yfinance.download()`.
Con alcune versioni di `yfinance` le colonne possono essere multi-livello o avere il nome del ticker attaccato (es. `Close_ELEC.F`): per questo facciamo un appiattimento e poi cerchiamo la colonna che inizia con `Close`.

In [ ]:
print("Scarico dati da Yahoo Finance...")
px = yf.download(YF_TICKER, start=START_DATE, end=END_DATE)

# appiattimento eventuale
if isinstance(px.columns, pd.MultiIndex):
    px.columns = ["_".join([c for c in col if c]).strip() for col in px.columns.values]

px = px.reset_index()

# prendiamo qualsiasi colonna che inizi con "Close"
price_cols = [c for c in px.columns if c.lower().startswith("close")]
if not price_cols:
    raise ValueError(f"Non trovo colonne di prezzo, colonne trovate: {px.columns}")
price_col = price_cols[0]

px = px.rename(columns={"Date": "date", price_col: "price"})
px["date"] = pd.to_datetime(px["date"]).dt.date

px.head()

## 4. Download dell'irradiazione solare da NASA POWER
Usiamo l'endpoint daily della NASA POWER API. Il parametro che ci interessa è `ALLSKY_SFC_SW_DWN` (Global Horizontal Irradiance). È obbligatorio anche il parametro `community`, qui impostato a `RE` (renewable energy).

In [ ]:
print("Scarico dati da NASA POWER...")
nasa_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
params = {
    "parameters": "ALLSKY_SFC_SW_DWN",
    "start": START_DATE.replace("-", ""),
    "end": END_DATE.replace("-", ""),
    "latitude": LAT,
    "longitude": LON,
    "community": "RE",
    "format": "JSON",
}
resp = requests.get(nasa_url, params=params)
resp.raise_for_status()
data_json = resp.json()

irr_dict = data_json["properties"]["parameter"]["ALLSKY_SFC_SW_DWN"]
irr_df = pd.DataFrame(list(irr_dict.items()), columns=["date", "ghi"])
irr_df["date"] = pd.to_datetime(irr_df["date"]).dt.date

irr_df.head()

## 5. Merge dei due dataset
Unifichiamo per data (inner join). In questo modo lavoriamo solo sui giorni per cui abbiamo **entrambe** le informazioni.
Se il merge è vuoto, vuol dire che periodo o coordinate non corrispondono.

In [ ]:
print("Faccio il merge dei dati...")
df = pd.merge(px[["date", "price"]], irr_df, on="date", how="inner").sort_values("date")
df = df.dropna(subset=["price", "ghi"])
print("Osservazioni disponibili:", len(df))
df.head()

## 6. Stima OLS
Il modello è:
\[ \text{price}_t = \beta_0 + \beta_1 \cdot \text{ghi}_t + \varepsilon_t. \]
Usiamo `statsmodels` per avere il riepilogo completo (stima, stderr, $R^2$, significatività, ecc.).

In [ ]:
X = sm.add_constant(df["ghi"])  # aggiunge la colonna di 1 per l'intercetta
y = df["price"]

model = sm.OLS(y, X).fit()
print(model.summary())

## 7. Visualizzazione: scatter + retta di regressione
Per vedere se la relazione è ragionevolmente lineare, facciamo uno scatter e ci sovrapponiamo la retta stimata. Ordiniamo i dati per `ghi` così la linea è pulita.

In [ ]:
print("Creo il plot …")
plt.figure(figsize=(10, 6))
plt.scatter(df["ghi"], df["price"], alpha=0.6, label="Dati")

# ordiniamo per ghi per la linea
df_sorted = df.sort_values("ghi")
y_hat = model.params["const"] + model.params["ghi"] * df_sorted["ghi"]

plt.plot(df_sorted["ghi"], y_hat, label="Regressione OLS", linewidth=2)

plt.xlabel("Irradiazione giornaliera (kWh/m²/day)")
plt.ylabel("Prezzo elettricità (proxy Yahoo)")
plt.title("Prezzo vs Irradiazione solare (Padova, 2024) – OLS")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 8. Note conclusive
- il ticker usato è solo un **proxy** del prezzo elettrico italiano;
- l'irradiazione è presa su una sola località (Padova);
- il modello è volutamente semplice (una sola variabile esplicativa) ed è pensato per mostrare la pipeline: **raccolta dati → merge → OLS → plot**;
- per un'analisi più realistica andrebbero aggiunte variabili di controllo e dummy stagionali.